# Wisdom of the Tribe 
## Database Design
### *Justin M. Olds* [github.com/jmolds](https://github.com/jmolds)
---
**Project Overview:** The ultimate purpose of this project is to develop a recommendation system for films based the match between a user's film preferences and the preferences of established film critics. This *Wisdom of the Tribes* approach contrasts with *Wisdom of the Masses* approaches provided by many popular websites, such as Metacritic and RottenTomatoes. 

---
In this notebook, I showcase how specific film and review information is scraped from metacritic.com and saved within a SQL database. Webpages were previously saved and this approach was detailed in another notebook: [saving-reviews-and-film-info-from-metacritic](https://github.com/jmolds/widsom-of-the-tribe/blob/master/saving-reviews-and-film-info-from-metacritic.ipynb "Notebook Link").

In [1]:
%load_ext blackcellmagic

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import IPython
from IPython.display import HTML
from IPython.display import display
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import csv
import pandas as pd
import numpy as np
import shelve  ### shalve was used in the previous notebook to save all

# html pages to be reinstated
r = shelve.open("films.requests.dat")
film_pages = r["film_pages"]
review_pages = r["review_pages"]
error_index = r["error_index"]
r.close()


## Database table creation
The schema for saving film and review information is show below:
![Example Advanced Search page](https://github.com/jmolds/widsom-of-the-tribe/blob/master/data-images-etc/database%20design.png?raw=true)

In [3]:
###create DB tables
import sqlite3
import pandas as pd
from datetime import datetime

conn = sqlite3.connect("films_and_reviews.db")
c = conn.cursor()

####### create tables
c.execute("DROP TABLE IF EXISTS film")
c.execute("DROP TABLE IF EXISTS review")
c.execute("DROP TABLE IF EXISTS author")
c.execute("DROP TABLE IF EXISTS genre")
c.execute("DROP TABLE IF EXISTS film_genre")

c.execute(
    """
          CREATE TABLE film (
          film_id INTEGER PRIMARY KEY AUTOINCREMENT, 
          film_title TEXT, 
          film_release_date TEXT,
          film_runtime INTEGER
          )"""
)

c.execute(
    """
          CREATE TABLE genre (
          genre_id INTEGER PRIMARY KEY AUTOINCREMENT, 
          genre_label TEXT 
          )"""
)

c.execute(
    """
          CREATE TABLE film_genre (
          film_id REFERENCES film(film_id),
          genre_id REFERENCES review(genre_id), 
              CONSTRAINT film_genre_pk PRIMARY KEY (film_id, genre_id)
          )"""
)

c.execute(
    """
          CREATE TABLE review (
          film_id INTEGER, 
          author_id INTEGER,
          rating INTEGER,
              FOREIGN KEY (film_id) REFERENCES film(film_id)
              FOREIGN KEY (author_id) REFERENCES author(author_id)
              CONSTRAINT review_pk PRIMARY KEY (film_id, author_id)
          )"""
)

c.execute(
    """
          CREATE TABLE author (
          author_id INTEGER PRIMARY KEY AUTOINCREMENT,
          first_name TEXT, 
          last_name TEXT
          )"""
)
conn.commit()


---
Next, functions for parsing film and review information were defined to make the code for looping through each film more readable.

In [5]:
### Define functions for webscraping
def scrape_film_title(soup_source):
    scrape = soup_source.select("div > h1")
    scrape = str(scrape)
    scrape = scrape.partition("</h1>")[0]
    scrape = scrape.partition("<h1>")[2]
    return scrape
####returns string to be temporarily saved for db insert

def scrape_film_release_date(soup_source):
    scrape = soup_source.select("span.release_date")
    scrape = str(scrape)
    scrape = scrape.partition("</span>\n<span>")[2]
    scrape = scrape.partition("</span>")[0]
    scrape = datetime.strptime(scrape, "%B %d, %Y")
    scrape = scrape.strftime("%Y-%m-%d")
    return scrape

def scrape_film_runtime(soup_source):
    scrape = soup_source.select("div.runtime")
    scrape = str(scrape)
    scrape = scrape.partition("</span>\n<span>")[2]
    scrape = scrape.partition(" min</span>")[0]
    return scrape

def scrape_film_genre_list(soup_source):
    scrape = soup_source.select("div.genres > span")[1]
    scrape = scrape.select("span")
    for x in range(0, len(scrape)):
        scrape[x] = str(scrape[x])
        scrape[x] = scrape[x].partition("</span>")[0]
        scrape[x] = scrape[x].partition("<span>")[2]
    return scrape
##### Returns a list of strings. List length can vary depending on the number of listed genres

def scrape_reviews_auth_name_list(soup_source):
    scrape = soup_source.select("span.author")
    first_name_list = [None] * len(scrape)
    last_name_list = [None] * len(scrape)
    last_name_first_name_list = [None] * len(scrape)
    for x in range(0, len(scrape)):
        scrape[x] = str(scrape[x])
        scrape[x] = scrape[x].partition('author">')[2]
        if "<a href=" in scrape[x]:
            scrape[x] = scrape[x].partition('movies">')[2]
            scrape[x] = scrape[x].partition("</a></span>")[0]
        else:
            scrape[x] = scrape[x].partition("</span>")[0]
        if "Staff" not in scrape[x]:
            first_name_list[x] = scrape[x].partition(" ")[0]
            last_name_list[x] = scrape[x].partition(" ")[2]
            last_name_first_name_list[x] = (last_name_list[x], first_name_list[x])
        else:
            last_name_first_name_list[x] = None
    return last_name_first_name_list
# Database NOTE: authors with middle names listed are included within Last_name
# (e.g., "Joy - Gould Boyum")
# Returns tuple for last name and first name or None object if author is uncredited

def scrape_reviews_rating_list(soup_source):
    scrape = soup_source.select("div.left.fl")
    for x in range(0, len(scrape)):
        scrape[x] = str(scrape[x])
        scrape[x] = scrape[x].partition("</div>")[0]
        scrape[x] = scrape[x].partition(">")[2]
        scrape[x] = scrape[x].partition(">")[2]
    return scrape


In [6]:
###Loop for extracting attributes and inserting into db
release_date_error = list()
runtime_error = list()
genre_film_unique_error = list()
for x in tqdm(range(0, len(film_pages))):
    ##Reset temp_variables
    temp_film_title = None
    temp_film_release_date = None
    temp_film_runtime = None
    temp_film_genre_list = None
    temp_reviews_name_list = None
    temp_reviews_ratings_list = None

    if film_pages[x] is not None:
        ## Parse film html for webscraping
        film_soup = BeautifulSoup(film_pages[x].content, "html.parser")
        if len(film_soup) > 0:
            ## Scrape title, release date and runtime and temporarily save for film x
            temp_film_title = scrape_film_title(film_soup)
            try:
                temp_film_release_date = scrape_film_release_date(film_soup)
            except:
                temp_film_release_date = "NULL"
                error_index.append(x)
            try:
                temp_film_runtime = scrape_film_runtime(film_soup)
            except:
                temp_film_runtime = "NULL"
                runtime_error.append(x)
            ## Insert film information into film table (no uniqueness checks required)
            c.execute(
                """
                  INSERT INTO film
                  (film_title, film_release_date, film_runtime) 
                  VALUES (?,?,?)""",
                [temp_film_title, temp_film_release_date, temp_film_runtime],
            )
            ## Scrape the list of genre labels for film x
            temp_film_genre_list = scrape_film_genre_list(film_soup)
            ## check if each genre label exists and insert any new labels into the genre table
            for i in range(
                0, len(temp_film_genre_list)
            ):  # NOTE: genre i refers to current genre
                c.execute(
                    "SELECT * FROM genre WHERE genre_label=?", [temp_film_genre_list[i]]
                )
                db_check_genre = c.fetchone()
                ## If genre label doesn't exist yet insert it into the genre table
                if db_check_genre is None:
                    c.execute(
                        "INSERT INTO genre (genre_label) VALUES (?)",
                        [temp_film_genre_list[i]],
                    )
                ## For updating the film_genre table -- retrieve film_id
                c.execute(
                    "SELECT film_id FROM film WHERE film_title=? AND film_release_date=?",
                    [temp_film_title, temp_film_release_date],
                )
                db_get_film_id = c.fetchone()
                ## For updating the film_genre table -- retrieve genre_id
                c.execute(
                    "SELECT genre_id FROM genre WHERE genre_label=?",
                    [temp_film_genre_list[i]],
                )
                db_get_genre_id = c.fetchone()
                ## Update the film_genre table -- for each genre i and film x
                try:
                    c.execute(
                        """
                              INSERT INTO film_genre
                              (film_id, genre_id) 
                              VALUES (?,?)""",
                        [db_get_film_id[0], db_get_genre_id[0]],
                    )
                except:
                    genre_film_unique_error.append(x)
            ## Parse reviews html for webscraping
            reviews_soup = BeautifulSoup(review_pages[x].content, "html.parser")
            ## Scrape author names and ratings for each review of film x
            temp_reviews_name_list = scrape_reviews_auth_name_list(reviews_soup)
            temp_reviews_ratings_list = scrape_reviews_rating_list(reviews_soup)
            ##For each review check if author name exists in author table and update table if not
            for j in range(
                0, len(temp_reviews_name_list)
            ):  # NOTE: name j refers to current reviewer name
                ## If the review is credited to an author
                if temp_reviews_name_list[j] is not None:
                    c.execute(
                        "SELECT * FROM author WHERE last_name=? AND first_name=?",
                        temp_reviews_name_list[j],
                    )
                    db_check_author = c.fetchone()
                    ## If author j doesn't exist yet insert it into the author table
                    if db_check_author is None:
                        c.execute(
                            "INSERT INTO author (last_name, first_name) VALUES (?,?)",
                            temp_reviews_name_list[j],
                        )
                    ## For updating the review table -- retrieve film_id
                    c.execute(
                        "SELECT film_id FROM film WHERE film_title=? AND film_release_date=?",
                        [temp_film_title, temp_film_release_date],
                    )
                    db_get_film_id = c.fetchone()
                    ## For updating the review table -- retrieve author_id
                    c.execute(
                        "SELECT author_id FROM author WHERE last_name=? AND first_name=?",
                        temp_reviews_name_list[j],
                    )
                    db_get_author_id = c.fetchone()
                    ## update review table for review j and film x
                    c.execute(
                        """
                        INSERT OR IGNORE INTO review
                        (film_id, author_id, rating) 
                        VALUES (?,?,?)""",
                        [
                            db_get_film_id[0],
                            db_get_author_id[0],
                            temp_reviews_ratings_list[j],
                        ],
                    )
                ## If the review is NOT credited to an author
                else:
                    ## create a new author_id for the uncredited review author
                    c.execute(
                        "INSERT INTO author (last_name, first_name) VALUES (?,?)",
                        ("Uncredited", "Uncredited"),
                    )
                    ## For updating the review table -- retrieve film_id
                    c.execute(
                        "SELECT film_id FROM film WHERE film_title=? AND film_release_date=?",
                        [temp_film_title, temp_film_release_date],
                    )
                    db_get_film_id = c.fetchone()
                    ## For updating the review table -- retrieve author_id
                    c.execute("SELECT MAX(author_id) FROM author")
                    db_get_author_id = c.fetchone()


### Inspect database

In [20]:
import sqlite3
import pandas as pd
conn = sqlite3.connect("C:/Users/Justin/Desktop/Projects/wisdom-of-the-tribe/films_and_reviews.db")
c = conn.cursor()

In [24]:
df = pd.read_sql_query("""
    SELECT *
    FROM genre
""", conn);df

,genre_id,genre_label
0,1,Action
1,2,Adventure
2,3,Sci-Fi
3,4,Romance
4,5,Comedy
5,6,Crime
6,7,War
7,8,Biography
8,9,Drama
9,10,Music


In [22]:
df = pd.read_sql_query("""
    SELECT *
    FROM film
""", conn);df

,film_id,film_title,film_release_date,film_runtime,film_rating_avg,film_rating_sd
0,1,Star Wars: Episode V - The Empire Strikes Back,1980-05-21,124,None,None
1,2,Superman II,1981-06-19,127,None,None
2,3,Stir Crazy,1980-12-12,111,None,None
3,4,Airplane!,1980-07-02,88,None,None
4,5,Any Which Way You Can,1980-12-17,116,None,None
5,6,Private Benjamin,1980-10-10,109,None,None
6,7,Coal Miner's Daughter,1980-03-07,124,None,None
7,8,Smokey and the Bandit II,1980-08-15,100,None,None
8,9,The Blue Lagoon,1980-07-02,104,None,None
9,10,The Blues Brothers,1980-06-20,133,None,None


In [23]:
df = pd.read_sql_query("""
    SELECT *
    FROM author
""", conn);df

,author_id,first_name,last_name,author_rating_avg,author_rating_sd,total_score,score_count,pref_aff,review_count
0,1,Charles,Champlin,None,None,None,None,None,None
1,2,Joe,Pollack,None,None,None,None,None,None
2,3,Uncredited,Uncredited,None,None,None,None,None,None
3,4,Gary,Arnold,None,None,None,None,None,None
4,5,Roger,Ebert,None,None,None,None,None,None
5,6,James,Berardinelli,None,None,None,None,None,None
6,7,Peter,Stack,None,None,None,None,None,None
7,8,Gene,Siskel,None,None,None,None,None,None
8,9,David,Sterritt,None,None,None,None,None,None
9,10,Lisa,Schwarzbaum,None,None,None,None,None,None


In [25]:
df = pd.read_sql_query("""
    SELECT *
    FROM review
""", conn);df

,film_id,author_id,rating
0,1,1,100
1,1,2,100
2,1,4,100
3,1,5,100
4,1,6,100
5,1,7,100
6,1,8,100
7,1,9,100
8,1,10,100
9,1,11,100
